In [1]:
pip install xgboost lightgbm catboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 30.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 25.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm import tqdm

from functools import partial
import scipy as sp

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor 
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/PS-S3/Ep9/train.csv'
file_key_2 = 'Tabular-Playground-Series/PS-S3/Ep9/test.csv'
file_key_3 = 'Tabular-Playground-Series/PS-S3/Ep9/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

Matplotlib is building the font cache; this may take a moment.


In [4]:
X = train.drop(columns = ['id', 'Strength'], axis = 1)
Y = train['Strength']
X['WaterComponent_to_Cement_ratio'] = X['WaterComponent'] / (X['CementComponent'] + 1e-6)

test_baseline = test.drop(columns = ['id'], axis = 1)
test_baseline['WaterComponent_to_Cement_ratio'] = test_baseline['WaterComponent'] / (test_baseline['CementComponent'] + 1e-6)


cat_md = CatBoostRegressor(loss_function = 'RMSE',
                           iterations = 1000,
                           learning_rate = 0.01,
                           depth = 3,
                           random_strength = 0.5,
                           bagging_temperature = 0.7,
                           border_count = 30,
                           l2_leaf_reg = 5,
                           verbose = False).fit(X, Y)

cat_pred = cat_md.predict(test_baseline)
submission['Strength'] = cat_pred
submission.head()

,id,Strength
0,5407,47.949070
1,5408,19.403589
2,5409,33.624952
3,5410,46.432587
4,5411,32.492635


In [5]:
submission.to_csv('catboost_full_submission.csv', index = False)

In [59]:
X = train.drop(columns = ['id', 'Strength'], axis = 1)
Y = train['Strength']
X['WaterComponent_to_Cement_ratio'] = X['WaterComponent'] / (X['CementComponent'] + 1e-6)

test_baseline = test.drop(columns = ['id'], axis = 1)
test_baseline['WaterComponent_to_Cement_ratio'] = test_baseline['WaterComponent'] / (test_baseline['CementComponent'] + 1e-6)

preds_train, preds_test = list(), list()

for i in tqdm(range(0, 1000)):
    
    CatBoostRegressor(loss_function = 'RMSE',
                           iterations = 1000,
                           learning_rate = 0.01,
                           depth = 3,
                           random_strength = 0.5,
                           bagging_temperature = 0.7,
                           border_count = 30,
                           l2_leaf_reg = 5,
                           verbose = False, 
                           random_seed = i).fit(X, Y)
    
#     cat_md = CatBoostRegressor(loss_function = 'RMSE',
#                                iterations = 1756,
#                                learning_rate = 0.017975994415651907,
#                                depth = 3,
#                                random_strength = 2.8219170899365174,
#                                bagging_temperature = 0.19623966374044916,
#                                border_count = 70,
#                                l2_leaf_reg = 29,
#                                verbose = False,
#                                random_seed = i).fit(X, Y)
    
    preds_train.append(cat_md.predict(X))
    preds_test.append(cat_md.predict(test_baseline))

100%|██████████| 1000/1000 [13:31<00:00,  1.23it/s]


In [60]:
cat_preds_test = pd.DataFrame(preds_test).apply(np.mean, axis = 0)

submission['Strength'] = cat_preds_test
submission.head()

,id,Strength
0,5407,48.030337
1,5408,18.512752
2,5409,33.690971
3,5410,48.222575
4,5411,28.180064


In [61]:
submission.to_csv('catboost_full_1000_submission.csv', index = False)

In [54]:
class OptimizedEnsemble(object):
    
    def __init__(self):
        self.coef_ = 0

    def _rmse_loss(self, coef, X, y):
        
        n = X.shape[1]
        ens = 0
        for i in range(0, n):
            ens += coef[i]*X[:, i]
        
        ll = mean_squared_error(y, ens, squared = False)
        return ll

    def fit(self, X, y):
        loss_partial = partial(self._rmse_loss, X = X, y = y)
        n = X.shape[1]
        initial_coef = np.repeat(1/n, n)
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        
        n = X.shape[1]
        ens = 0
        for i in range(0, n):
            ens += coef[i]*X[:, i]
        return ens

    def coefficients(self):
        return self.coef_['x']

In [55]:
models_pred_train = np.array(pd.DataFrame(preds_train).T)
models_pred_oof = np.array(pd.DataFrame(preds_test).T)

# n = models_pred_oof.shape[1]
# ens = 0
# for i in range(0, n):
#     ens += models_pred_oof[:, i]

opt_ens = OptimizedEnsemble()
opt_ens.fit(models_pred_train, Y)
coef = opt_ens.coefficients()
ens_pred_train = opt_ens.predict(models_pred_train, coef)
ens_pred_test = opt_ens.predict(models_pred_oof, coef)        

In [56]:
mean_squared_error(Y, ens_pred_train, squared = False)

11.542104753650678

In [57]:
sum(coef)

1.0020166053882114

In [58]:
submission['Strength'] = ens_pred_test
submission.head()

,id,Strength
0,5407,49.647288
1,5408,16.885482
2,5409,34.435151
3,5410,48.597436
4,5411,23.453819


In [45]:
submission.to_csv('catboost_full_50_opt_ens_submission.csv', index = False)

In [18]:
np.array(pd.DataFrame(preds).T).shape

(3605, 50)

In [ ]:
X = train.drop(columns = ['id', 'Strength'], axis = 1)
Y = train['Strength']

test_baseline = test.drop(columns = ['id'], axis = 1)

XGB_cv_scores, XGB_imp = list(), list()
XGB_preds = list()

lgb_cv_scores, lgb_imp = list(), list()
lgb_preds = list()

cat_cv_scores, cat_imp = list(), list()
cat_preds = list()

for i in range(5):

    skf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        #############    
        ## XGBoost ##
        #############
        
        XGB_md = XGBRegressor(tree_method = 'hist',
                              colsample_bytree = 0.7, 
                              gamma = 0.8, 
                              learning_rate = 0.01, 
                              max_depth = 7, 
                              min_child_weight = 10, 
                              n_estimators = 1000, 
                              subsample = 0.7).fit(X_train, Y_train)
        XGB_imp.append(XGB_md.feature_importances_)
        
        ## Predicting on X_test and test
        XGB_pred_1 = XGB_md.predict(X_test)
        XGB_pred_2 = XGB_md.predict(test_baseline)
        
        ## Computing rmse
        XGB_cv_scores.append(mean_squared_error(Y_test, XGB_pred_1, squared = False))
        XGB_preds.append(XGB_pred_2)
        
        ##############
        ## LightGBM ##
        ##############
        
        lgb_md = LGBMRegressor(n_estimators = 1000,
                               max_depth = 7,
                               learning_rate = 0.01,
                               num_leaves = 20,
                               lambda_l1 = 3,
                               lambda_l2 = 3,
                               bagging_fraction = 0.7,
                               feature_fraction = 0.7).fit(X_train, Y_train)
        lgb_imp.append(lgb_md.feature_importances_)
        
        ## Predicting on X_test and test
        lgb_pred_1 = lgb_md.predict(X_test)
        lgb_pred_2 = lgb_md.predict(test_baseline)
        
        ## Computing rmse
        lgb_cv_scores.append(mean_squared_error(Y_test, lgb_pred_1, squared = False))
        lgb_preds.append(lgb_pred_2)
        
        ##############
        ## CatBoost ##
        ##############
        
        cat_md = CatBoostRegressor(loss_function = 'RMSE',
                                   iterations = 1000,
                                   learning_rate = 0.01,
                                   depth = 7,
                                   random_strength = 0.5,
                                   bagging_temperature = 0.7,
                                   border_count = 30,
                                   l2_leaf_reg = 5,
                                   verbose = False).fit(X_train, Y_train)
        cat_imp.append(cat_md.feature_importances_)
        
        ## Predicting on X_test and test
        cat_pred_1 = cat_md.predict(X_test)
        cat_pred_2 = cat_md.predict(test_baseline)
        
        ## Computing rmse
        cat_cv_scores.append(mean_squared_error(Y_test, cat_pred_1, squared = False))
        cat_preds.append(cat_pred_2)

# Baseline Modeling 1.0

In [ ]:
X = train.drop(columns = ['id', 'Strength'], axis = 1)
Y = train['Strength']

test_baseline = test.drop(columns = ['id'], axis = 1)

XGB_cv_scores, XGB_imp = list(), list()
XGB_preds = list()

lgb_cv_scores, lgb_imp = list(), list()
lgb_preds = list()

cat_cv_scores, cat_imp = list(), list()
cat_preds = list()

for i in range(5):

    skf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        #############    
        ## XGBoost ##
        #############
        
        XGB_md = XGBRegressor(tree_method = 'hist',
                              colsample_bytree = 0.7, 
                              gamma = 0.8, 
                              learning_rate = 0.01, 
                              max_depth = 7, 
                              min_child_weight = 10, 
                              n_estimators = 1000, 
                              subsample = 0.7).fit(X_train, Y_train)
        XGB_imp.append(XGB_md.feature_importances_)
        
        ## Predicting on X_test and test
        XGB_pred_1 = XGB_md.predict(X_test)
        XGB_pred_2 = XGB_md.predict(test_baseline)
        
        ## Computing rmse
        XGB_cv_scores.append(mean_squared_error(Y_test, XGB_pred_1, squared = False))
        XGB_preds.append(XGB_pred_2)
        
        ##############
        ## LightGBM ##
        ##############
        
        lgb_md = LGBMRegressor(n_estimators = 1000,
                               max_depth = 7,
                               learning_rate = 0.01,
                               num_leaves = 20,
                               lambda_l1 = 3,
                               lambda_l2 = 3,
                               bagging_fraction = 0.7,
                               feature_fraction = 0.7).fit(X_train, Y_train)
        lgb_imp.append(lgb_md.feature_importances_)
        
        ## Predicting on X_test and test
        lgb_pred_1 = lgb_md.predict(X_test)
        lgb_pred_2 = lgb_md.predict(test_baseline)
        
        ## Computing rmse
        lgb_cv_scores.append(mean_squared_error(Y_test, lgb_pred_1, squared = False))
        lgb_preds.append(lgb_pred_2)
        
        ##############
        ## CatBoost ##
        ##############
        
        cat_md = CatBoostRegressor(loss_function = 'RMSE',
                                   iterations = 1000,
                                   learning_rate = 0.01,
                                   depth = 7,
                                   random_strength = 0.5,
                                   bagging_temperature = 0.7,
                                   border_count = 30,
                                   l2_leaf_reg = 5,
                                   verbose = False).fit(X_train, Y_train)
        cat_imp.append(cat_md.feature_importances_)
        
        ## Predicting on X_test and test
        cat_pred_1 = cat_md.predict(X_test)
        cat_pred_2 = cat_md.predict(test_baseline)
        
        ## Computing rmse
        cat_cv_scores.append(mean_squared_error(Y_test, cat_pred_1, squared = False))
        cat_preds.append(cat_pred_2)

In [ ]:
XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the XGBoost model is:', XGB_cv_score)

lgb_cv_score = np.mean(lgb_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the LightGBM model is:', lgb_cv_score)

cat_cv_score = np.mean(cat_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is:', cat_cv_score)

In [ ]:
cat_preds_test = pd.DataFrame(cat_preds).apply(np.mean, axis = 0)

submission['Strength'] = cat_preds_test
submission.head()

In [ ]:
submission.to_csv('catboost_submission.csv', index = False)

# Baseline Modeling 2.0

In [ ]:
train_no_dup = train.drop(columns = 'id', axis = 1)
train_no_dup = pd.DataFrame(train_no_dup.groupby(train_no_dup.columns.tolist()[0:8])['Strength'].mean()).reset_index()

X = train_no_dup.drop(columns = ['Strength'], axis = 1)
Y = train_no_dup['Strength']

test_baseline = test.drop(columns = ['id'], axis = 1)

XGB_cv_scores, XGB_imp = list(), list()
XGB_preds = list()

lgb_cv_scores, lgb_imp = list(), list()
lgb_preds = list()

cat_cv_scores, cat_imp = list(), list()
cat_preds = list()

for i in range(5):

    skf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        #############    
        ## XGBoost ##
        #############
        
        XGB_md = XGBRegressor(tree_method = 'hist',
                              colsample_bytree = 0.7, 
                              gamma = 0.8, 
                              learning_rate = 0.01, 
                              max_depth = 7, 
                              min_child_weight = 10, 
                              n_estimators = 1000, 
                              subsample = 0.7).fit(X_train, Y_train)
        XGB_imp.append(XGB_md.feature_importances_)
        
        ## Predicting on X_test and test
        XGB_pred_1 = XGB_md.predict(X_test)
        XGB_pred_2 = XGB_md.predict(test_baseline)
        
        ## Computing rmse
        XGB_cv_scores.append(mean_squared_error(Y_test, XGB_pred_1, squared = False))
        XGB_preds.append(XGB_pred_2)
        
        ##############
        ## LightGBM ##
        ##############
        
        lgb_md = LGBMRegressor(n_estimators = 1000,
                               max_depth = 7,
                               learning_rate = 0.01,
                               num_leaves = 20,
                               lambda_l1 = 3,
                               lambda_l2 = 3,
                               bagging_fraction = 0.7,
                               feature_fraction = 0.7).fit(X_train, Y_train)
        lgb_imp.append(lgb_md.feature_importances_)
        
        ## Predicting on X_test and test
        lgb_pred_1 = lgb_md.predict(X_test)
        lgb_pred_2 = lgb_md.predict(test_baseline)
        
        ## Computing rmse
        lgb_cv_scores.append(mean_squared_error(Y_test, lgb_pred_1, squared = False))
        lgb_preds.append(lgb_pred_2)
        
        ##############
        ## CatBoost ##
        ##############
        
        cat_md = CatBoostRegressor(loss_function = 'RMSE',
                                   iterations = 1000,
                                   learning_rate = 0.01,
                                   depth = 7,
                                   random_strength = 0.5,
                                   bagging_temperature = 0.7,
                                   border_count = 30,
                                   l2_leaf_reg = 5,
                                   verbose = False).fit(X_train, Y_train)
        cat_imp.append(cat_md.feature_importances_)
        
        ## Predicting on X_test and test
        cat_pred_1 = cat_md.predict(X_test)
        cat_pred_2 = cat_md.predict(test_baseline)
        
        ## Computing rmse
        cat_cv_scores.append(mean_squared_error(Y_test, cat_pred_1, squared = False))
        cat_preds.append(cat_pred_2)

In [ ]:
XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the XGBoost model is:', XGB_cv_score)

lgb_cv_score = np.mean(lgb_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the LightGBM model is:', lgb_cv_score)

cat_cv_score = np.mean(cat_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is:', cat_cv_score)

In [ ]:
cat_preds_test = pd.DataFrame(cat_preds).apply(np.mean, axis = 0)

submission['Strength'] = cat_preds_test
submission.head()

In [ ]:
submission.to_csv('catboost_submission_1.csv', index = False)

In [5]:
train_no_dup = train.drop(columns = 'id', axis = 1)
train_no_dup = pd.DataFrame(train_no_dup.groupby(train_no_dup.columns.tolist()[0:8])['Strength'].median()).reset_index()

X = train_no_dup.drop(columns = ['Strength'], axis = 1)
Y = train_no_dup['Strength']

test_baseline = test.drop(columns = ['id'], axis = 1)

XGB_cv_scores, XGB_imp = list(), list()
XGB_preds = list()

lgb_cv_scores, lgb_imp = list(), list()
lgb_preds = list()

cat_cv_scores, cat_imp = list(), list()
cat_preds = list()

for i in range(5):

    skf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        #############    
        ## XGBoost ##
        #############
        
#         XGB_md = XGBRegressor(tree_method = 'hist',
#                               colsample_bytree = 0.7, 
#                               gamma = 0.8, 
#                               learning_rate = 0.01, 
#                               max_depth = 7, 
#                               min_child_weight = 10, 
#                               n_estimators = 1000, 
#                               subsample = 0.7).fit(X_train, Y_train)
#         XGB_imp.append(XGB_md.feature_importances_)
        
#         ## Predicting on X_test and test
#         XGB_pred_1 = XGB_md.predict(X_test)
#         XGB_pred_2 = XGB_md.predict(test_baseline)
        
#         ## Computing rmse
#         XGB_cv_scores.append(mean_squared_error(Y_test, XGB_pred_1, squared = False))
#         XGB_preds.append(XGB_pred_2)
        
        ##############
        ## LightGBM ##
        ##############
        
#         lgb_md = LGBMRegressor(n_estimators = 1000,
#                                max_depth = 7,
#                                learning_rate = 0.01,
#                                num_leaves = 20,
#                                lambda_l1 = 3,
#                                lambda_l2 = 3,
#                                bagging_fraction = 0.7,
#                                feature_fraction = 0.7).fit(X_train, Y_train)
#         lgb_imp.append(lgb_md.feature_importances_)
        
#         ## Predicting on X_test and test
#         lgb_pred_1 = lgb_md.predict(X_test)
#         lgb_pred_2 = lgb_md.predict(test_baseline)
        
#         ## Computing rmse
#         lgb_cv_scores.append(mean_squared_error(Y_test, lgb_pred_1, squared = False))
#         lgb_preds.append(lgb_pred_2)
        
        ##############
        ## CatBoost ##
        ##############
        
        cat_md = CatBoostRegressor(loss_function = 'RMSE',
                                   iterations = 4738,
                                   learning_rate = 0.003143666241424718,
                                   depth = 4,
                                   random_strength = 0.29823973415192867,
                                   bagging_temperature = 0.3408793603898661,
                                   border_count = 112,
                                   l2_leaf_reg = 17,
                                   verbose = False).fit(X_train, Y_train)
        cat_imp.append(cat_md.feature_importances_)
        
        ## Predicting on X_test and test
        cat_pred_1 = cat_md.predict(X_test)
        cat_pred_2 = cat_md.predict(test_baseline)
        
        ## Computing rmse
        cat_cv_scores.append(mean_squared_error(Y_test, cat_pred_1, squared = False))
        cat_preds.append(cat_pred_2)

In [6]:
# XGB_cv_score = np.mean(XGB_cv_scores)    
# print('The average oof rmse score over 5-folds (run 5 times) of the XGBoost model is:', XGB_cv_score)

# lgb_cv_score = np.mean(lgb_cv_scores)    
# print('The average oof rmse score over 5-folds (run 5 times) of the LightGBM model is:', lgb_cv_score)

cat_cv_score = np.mean(cat_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is:', cat_cv_score)

The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is: 11.880628569091348


# Baseline 3.0

In [2]:
class OptimizedEnsemble(object):
    
    def __init__(self):
        self.coef_ = 0

    def _rmse_loss(self, coef, X, y):
        
        ens = coef[0]*X[:, 0] + coef[1]*X[:, 1] + coef[2]*X[:, 2]
        ll = mean_squared_error(y, ens, squared = False)
        return ll

    def fit(self, X, y):
        loss_partial = partial(self._rmse_loss, X = X, y = y)
        initial_coef = [1/3, 1/3, 1/3]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        
        ens = coef[0]*X[:, 0] + coef[1]*X[:, 1] + coef[2]*X[:, 2]
        return ens

    def coefficients(self):
        return self.coef_['x']

In [3]:
train_no_dup = train.drop(columns = 'id', axis = 1)
train_no_dup = pd.DataFrame(train_no_dup.groupby(train_no_dup.columns.tolist()[0:8])['Strength'].median()).reset_index()

X = train_no_dup.drop(columns = ['Strength'], axis = 1)
Y = train_no_dup['Strength']

test_baseline = test.drop(columns = ['id'], axis = 1)

ens_cv_scores, preds = list(), list()

for i in tqdm(range(5)):

    skf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        #############    
        ## XGBoost ##
        #############
        
        XGB_md = XGBRegressor(tree_method = 'hist',
                              colsample_bytree = 0.671460244215802, 
                              gamma = 2.5281806276307384, 
                              learning_rate = 0.002046162779305807, 
                              max_depth = 8, 
                              min_child_weight = 80, 
                              n_estimators = 2690, 
                              subsample = 0.44886485549735244).fit(X_train, Y_train)
        
        ##############
        ## LightGBM ##
        ##############
        
        lgb_md = LGBMRegressor(n_estimators = 5420,
                               max_depth = 3,
                               learning_rate = 0.0014779400349972686,
                               num_leaves = 61,
                               lambda_l1 = 7.384172796287736,
                               lambda_l2 = 0.10456555506292783,
                               bagging_fraction = 0.22841166601766863,
                               feature_fraction = 0.659898030).fit(X_train, Y_train)
        
        ##############
        ## CatBoost ##
        ##############
        
        cat_md = CatBoostRegressor(loss_function = 'RMSE',
                                   iterations = 4738,
                                   learning_rate = 0.003143666241424718,
                                   depth = 4,
                                   random_strength = 0.29823973415192867,
                                   bagging_temperature = 0.3408793603898661,
                                   border_count = 112,
                                   l2_leaf_reg = 17,
                                   verbose = False).fit(X_train, Y_train)
        
        ######################
        ## Optimal Ensemble ##
        ######################
        
        XGB_pred_1 = XGB_md.predict(X_test)
        lgb_pred_1 = lgb_md.predict(X_test)
        cat_pred_1 = cat_md.predict(X_test)
        models_pred_oof = np.transpose((XGB_pred_1, lgb_pred_1, cat_pred_1))

        opt_ens = OptimizedEnsemble()
        opt_ens.fit(models_pred_oof, Y_test)
        coef = opt_ens.coefficients()
        
        ens_pred = opt_ens.predict(models_pred_oof, coef)
        ens_cv_scores.append(mean_squared_error(Y_test, ens_pred, squared = False))
        
        XGB_pred_2 = XGB_md.predict(test_baseline)
        lgb_pred_2 = lgb_md.predict(test_baseline)
        cat_pred_2 = cat_md.predict(test_baseline)
        models_pred = np.transpose((XGB_pred_2, lgb_pred_2, cat_pred_2))
        
        ens_preds = opt_ens.predict(models_pred, coef)
        preds.append(ens_preds)

  0%|          | 0/5 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=7.384172796287736, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.384172796287736
[LightGBM] [Warning] feature_fraction is set=0.65989803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65989803
[LightGBM] [Warning] lambda_l2 is set=0.10456555506292783, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.10456555506292783
[LightGBM] [Warning] bagging_fraction is set=0.22841166601766863, subsample=1.0 will be ignored. Current value: bagging_fraction=0.22841166601766863
[LightGBM] [Warning] lambda_l1 is set=7.384172796287736, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.384172796287736
[LightGBM] [Warning] feature_fraction is set=0.65989803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65989803
[LightGBM] [Warning] lambda_l2 is set=0.10456555506292783, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.10456555506292783
[LightGBM] [Warning] bagging_fraction is 

 20%|██        | 1/5 [02:27<09:50, 147.55s/it]

[LightGBM] [Warning] lambda_l1 is set=7.384172796287736, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.384172796287736
[LightGBM] [Warning] feature_fraction is set=0.65989803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65989803
[LightGBM] [Warning] lambda_l2 is set=0.10456555506292783, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.10456555506292783
[LightGBM] [Warning] bagging_fraction is set=0.22841166601766863, subsample=1.0 will be ignored. Current value: bagging_fraction=0.22841166601766863
[LightGBM] [Warning] lambda_l1 is set=7.384172796287736, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.384172796287736
[LightGBM] [Warning] feature_fraction is set=0.65989803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65989803
[LightGBM] [Warning] lambda_l2 is set=0.10456555506292783, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.10456555506292783
[LightGBM] [Warning] bagging_fraction is 

 40%|████      | 2/5 [04:48<07:11, 143.76s/it]

...

 60%|██████    | 3/5 [07:05<04:40, 140.41s/it]

[LightGBM] [Warning] lambda_l1 is set=7.384172796287736, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.384172796287736
[LightGBM] [Warning] feature_fraction is set=0.65989803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65989803
[LightGBM] [Warning] lambda_l2 is set=0.10456555506292783, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.10456555506292783
[LightGBM] [Warning] bagging_fraction is set=0.22841166601766863, subsample=1.0 will be ignored. Current value: bagging_fraction=0.22841166601766863
[LightGBM] [Warning] lambda_l1 is set=7.384172796287736, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.384172796287736
[LightGBM] [Warning] feature_fraction is set=0.65989803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65989803
[LightGBM] [Warning] lambda_l2 is set=0.10456555506292783, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.10456555506292783
[LightGBM] [Warning] bagging_fraction is 

 80%|████████  | 4/5 [09:13<02:15, 135.61s/it]

[LightGBM] [Warning] lambda_l1 is set=7.384172796287736, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.384172796287736
[LightGBM] [Warning] feature_fraction is set=0.65989803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65989803
[LightGBM] [Warning] lambda_l2 is set=0.10456555506292783, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.10456555506292783
[LightGBM] [Warning] bagging_fraction is set=0.22841166601766863, subsample=1.0 will be ignored. Current value: bagging_fraction=0.22841166601766863
[LightGBM] [Warning] lambda_l1 is set=7.384172796287736, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.384172796287736
[LightGBM] [Warning] feature_fraction is set=0.65989803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65989803
[LightGBM] [Warning] lambda_l2 is set=0.10456555506292783, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.10456555506292783
[LightGBM] [Warning] bagging_fraction is 

100%|██████████| 5/5 [11:35<00:00, 139.09s/it]


In [4]:
ens_cv_score = np.mean(ens_cv_scores) # first run    
print('The average oof rmse score over 5-folds (run 5 times) of the ensemble model is:', ens_cv_score)

The average oof rmse score over 5-folds (run 5 times) of the ensemble model is: 11.835470374627613


In [4]:
ens_cv_score = np.mean(ens_cv_scores)  # second run  
print('The average oof rmse score over 5-folds (run 5 times) of the ensemble model is:', ens_cv_score)

The average oof rmse score over 5-folds (run 5 times) of the ensemble model is: 11.835470374627613


In [ ]:
ens_preds_test = pd.DataFrame(preds).apply(np.mean, axis = 0)

submission['Strength'] = ens_preds_test
submission.to_csv('Ensemble_Optuna_baseline_submission.csv', index = False)

## Full Dataset

In [5]:
X.head()

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays
0,102.0,0.0,0.0,192.0,0.0,879.0,942.0,3
1,102.0,153.0,0.0,192.0,0.0,887.0,942.0,3
2,102.0,153.0,0.0,192.0,0.0,887.0,942.0,7
3,102.0,153.0,0.0,192.0,0.0,887.0,942.0,28
4,102.0,153.0,0.0,192.0,0.0,887.0,942.0,90


In [ ]:
train_no_dup = train.drop(columns = 'id', axis = 1)
train_no_dup = pd.DataFrame(train_no_dup.groupby(train_no_dup.columns.tolist()[0:8])['Strength'].median()).reset_index()

X = train_no_dup.drop(columns = ['Strength'], axis = 1)
Y = train_no_dup['Strength']

test_baseline = test.drop(columns = ['id'], axis = 1)

ens_cv_scores, preds = list(), list()

for i in tqdm(range(5)):

    skf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
#         #############    
#         ## XGBoost ##
#         #############
        
#         XGB_md = XGBRegressor(tree_method = 'hist',
#                               colsample_bytree = 0.671460244215802, 
#                               gamma = 2.5281806276307384, 
#                               learning_rate = 0.002046162779305807, 
#                               max_depth = 8, 
#                               min_child_weight = 80, 
#                               n_estimators = 2690, 
#                               subsample = 0.44886485549735244).fit(X_train, Y_train)
        
#         ##############
#         ## LightGBM ##
#         ##############
        
#         lgb_md = LGBMRegressor(n_estimators = 5420,
#                                max_depth = 3,
#                                learning_rate = 0.0014779400349972686,
#                                num_leaves = 61,
#                                lambda_l1 = 7.384172796287736,
#                                lambda_l2 = 0.10456555506292783,
#                                bagging_fraction = 0.22841166601766863,
#                                feature_fraction = 0.659898030).fit(X_train, Y_train)
        
        ##############
        ## CatBoost ##
        ##############
        
        cat_md = CatBoostRegressor(loss_function = 'RMSE',
                                   iterations = 4738,
                                   learning_rate = 0.003143666241424718,
                                   depth = 4,
                                   random_strength = 0.29823973415192867,
                                   bagging_temperature = 0.3408793603898661,
                                   border_count = 112,
                                   l2_leaf_reg = 17,
                                   verbose = False).fit(X_train, Y_train)
        
        ######################
        ## Optimal Ensemble ##
        ######################
        
        XGB_pred_1 = XGB_md.predict(X_test)
        lgb_pred_1 = lgb_md.predict(X_test)
        cat_pred_1 = cat_md.predict(X_test)
        models_pred_oof = np.transpose((XGB_pred_1, lgb_pred_1, cat_pred_1))

        opt_ens = OptimizedEnsemble()
        opt_ens.fit(models_pred_oof, Y_test)
        coef = opt_ens.coefficients()
        
        ens_pred = opt_ens.predict(models_pred_oof, coef)
        ens_cv_scores.append(mean_squared_error(Y_test, ens_pred, squared = False))
        
        XGB_pred_2 = XGB_md.predict(test_baseline)
        lgb_pred_2 = lgb_md.predict(test_baseline)
        cat_pred_2 = cat_md.predict(test_baseline)
        models_pred = np.transpose((XGB_pred_2, lgb_pred_2, cat_pred_2))
        
        ens_preds = opt_ens.predict(models_pred, coef)
        preds.append(ens_preds)

# Baseline 4.0

In [2]:
class OptimizedEnsemble(object):
    
    def __init__(self):
        self.coef_ = 0

    def _rmse_loss(self, coef, X, y):
        
        ens = coef[0]*X[:, 0] + coef[1]*X[:, 1] + coef[2]*X[:, 2]
        ll = mean_squared_error(y, ens, squared = False)
        return ll

    def fit(self, X, y):
        loss_partial = partial(self._rmse_loss, X = X, y = y)
        initial_coef = [1/3, 1/3, 1/3]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        
        ens = coef[0]*X[:, 0] + coef[1]*X[:, 1] + coef[2]*X[:, 2]
        return ens

    def coefficients(self):
        return self.coef_['x']

In [3]:
train_no_dup = train.drop(columns = 'id', axis = 1)
train_no_dup = pd.DataFrame(train_no_dup.groupby(train_no_dup.columns.tolist()[0:8])['Strength'].median()).reset_index()

X = train_no_dup.drop(columns = ['Strength'], axis = 1)
Y = train_no_dup['Strength']

X['WaterComponent_to_Cement_ratio'] = X['WaterComponent'] / (X['CementComponent'] + 1e-6)

test_baseline = test.drop(columns = ['id'], axis = 1)
test_baseline['WaterComponent_to_Cement_ratio'] = test_baseline['WaterComponent'] / (test_baseline['CementComponent'] + 1e-6)

ens_cv_scores, preds = list(), list()

for i in tqdm(range(5)):

    skf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        #############    
        ## XGBoost ##
        #############
        
        XGB_md = XGBRegressor(tree_method = 'hist',
                              colsample_bytree = 0.7940111565728297, 
                              gamma = 3.3316249893010292, 
                              learning_rate = 0.0009167011149361065, 
                              max_depth = 8, 
                              min_child_weight = 89, 
                              n_estimators = 6766, 
                              subsample = 0.3771654611184001).fit(X_train, Y_train)
        
        ##############
        ## LightGBM ##
        ##############
        
        lgb_md = LGBMRegressor(n_estimators = 4273,
                               max_depth = 4,
                               learning_rate = 0.002480759916271656,
                               num_leaves = 5,
                               lambda_l1 = 1.2582711296889206,
                               lambda_l2 = 0.018040323452000916,
                               bagging_fraction = 0.7658802307213928,
                               feature_fraction = 0.4895760311486668).fit(X_train, Y_train)
        
        ##############
        ## CatBoost ##
        ##############
        
        cat_md = CatBoostRegressor(loss_function = 'RMSE',
                                   iterations = 416,
                                   learning_rate = 0.031225761812299576,
                                   depth = 4,
                                   random_strength = 0.19789440193456237,
                                   bagging_temperature = 0.2831892755259466,
                                   border_count = 56,
                                   l2_leaf_reg = 26,
                                   verbose = False).fit(X_train, Y_train)
        
        ######################
        ## Optimal Ensemble ##
        ######################
        
        XGB_pred_1 = XGB_md.predict(X_test)
        lgb_pred_1 = lgb_md.predict(X_test)
        cat_pred_1 = cat_md.predict(X_test)
        models_pred_oof = np.transpose((XGB_pred_1, lgb_pred_1, cat_pred_1))

        opt_ens = OptimizedEnsemble()
        opt_ens.fit(models_pred_oof, Y_test)
        coef = opt_ens.coefficients()
        
        ens_pred = opt_ens.predict(models_pred_oof, coef)
        ens_cv_scores.append(mean_squared_error(Y_test, ens_pred, squared = False))
        
        XGB_pred_2 = XGB_md.predict(test_baseline)
        lgb_pred_2 = lgb_md.predict(test_baseline)
        cat_pred_2 = cat_md.predict(test_baseline)
        models_pred = np.transpose((XGB_pred_2, lgb_pred_2, cat_pred_2))
        
        ens_preds = opt_ens.predict(models_pred, coef)
        preds.append(ens_preds)

  0%|          | 0/5 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=1.2582711296889206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2582711296889206
[LightGBM] [Warning] feature_fraction is set=0.4895760311486668, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4895760311486668
[LightGBM] [Warning] lambda_l2 is set=0.018040323452000916, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018040323452000916
[LightGBM] [Warning] bagging_fraction is set=0.7658802307213928, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7658802307213928
[LightGBM] [Warning] lambda_l1 is set=1.2582711296889206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2582711296889206
[LightGBM] [Warning] feature_fraction is set=0.4895760311486668, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4895760311486668
[LightGBM] [Warning] lambda_l2 is set=0.018040323452000916, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018040323452000916
[Li

 20%|██        | 1/5 [00:24<01:37, 24.37s/it]

[LightGBM] [Warning] lambda_l1 is set=1.2582711296889206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2582711296889206
[LightGBM] [Warning] feature_fraction is set=0.4895760311486668, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4895760311486668
[LightGBM] [Warning] lambda_l2 is set=0.018040323452000916, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018040323452000916
[LightGBM] [Warning] bagging_fraction is set=0.7658802307213928, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7658802307213928
[LightGBM] [Warning] lambda_l1 is set=1.2582711296889206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2582711296889206
[LightGBM] [Warning] feature_fraction is set=0.4895760311486668, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4895760311486668
[LightGBM] [Warning] lambda_l2 is set=0.018040323452000916, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018040323452000916
[Li

 40%|████      | 2/5 [00:45<01:08, 22.69s/it]

...

 60%|██████    | 3/5 [01:12<00:48, 24.27s/it]

[LightGBM] [Warning] lambda_l1 is set=1.2582711296889206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2582711296889206
[LightGBM] [Warning] feature_fraction is set=0.4895760311486668, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4895760311486668
[LightGBM] [Warning] lambda_l2 is set=0.018040323452000916, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018040323452000916
[LightGBM] [Warning] bagging_fraction is set=0.7658802307213928, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7658802307213928
[LightGBM] [Warning] lambda_l1 is set=1.2582711296889206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2582711296889206
[LightGBM] [Warning] feature_fraction is set=0.4895760311486668, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4895760311486668
[LightGBM] [Warning] lambda_l2 is set=0.018040323452000916, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018040323452000916
[Li

 80%|████████  | 4/5 [01:37<00:24, 24.60s/it]

[LightGBM] [Warning] lambda_l1 is set=1.2582711296889206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2582711296889206
[LightGBM] [Warning] feature_fraction is set=0.4895760311486668, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4895760311486668
[LightGBM] [Warning] lambda_l2 is set=0.018040323452000916, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018040323452000916
[LightGBM] [Warning] bagging_fraction is set=0.7658802307213928, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7658802307213928
[LightGBM] [Warning] lambda_l1 is set=1.2582711296889206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2582711296889206
[LightGBM] [Warning] feature_fraction is set=0.4895760311486668, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4895760311486668
[LightGBM] [Warning] lambda_l2 is set=0.018040323452000916, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018040323452000916
[Li

100%|██████████| 5/5 [02:06<00:00, 25.23s/it]


In [4]:
ens_cv_score = np.mean(ens_cv_scores) 
print('The average oof rmse score over 5-folds (run 5 times) of the ensemble model is:', ens_cv_score)

The average oof rmse score over 5-folds (run 5 times) of the ensemble model is: 11.813790317617261


In [5]:
ens_preds_test = pd.DataFrame(preds).apply(np.mean, axis = 0)

submission['Strength'] = ens_preds_test
submission.to_csv('Ensemble_Optuna_baseline_submission_2.csv', index = False)

In [3]:
cat_cv_score = np.mean(cat_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is:', cat_cv_score)

The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is: 12.08682356233276


In [3]:
cat_cv_score = np.mean(cat_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is:', cat_cv_score)

The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is: 12.069039135775649


In [7]:
cat_cv_score = np.mean(cat_cv_scores)    
print('The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is:', cat_cv_score)

The average oof rmse score over 5-folds (run 5 times) of the CatBoost model is: 12.088690775686844


In [8]:
cat_preds_test = pd.DataFrame(cat_preds).apply(np.mean, axis = 0)

submission['Strength'] = cat_preds_test
submission.head()

,id,Strength
0,5407,47.415894
1,5408,19.540387
2,5409,33.601072
3,5410,46.470701
4,5411,32.705374


In [9]:
submission.to_csv('catboost_submission_6.csv', index = False)